**stations**

StationReference = Name | MayBeInitial | Platforms

|Name | MayBeInitial | Platforms |  
 |--|--|--|  
|a = 鄂州站 | 1 | 2, 1, 3, 6, 4, 5 |  
|b = 武九线黄石方向 | 1 | 2, 1 |  
|c = 武九客专黄石北方向 | 1 | 1, 2 |  
|d = 武九客专葛店南方向 | 1 | 2, 1 |  
|e = 武九线何刘方向 | 1 | 1, 2 |  

**timetable**  
 Format: 
 ReportingNumber TrainType MaxSpeedKmph TrainComposition Flags : StationVisit1 StationVisit2 ...  

 StationVisit format:  
 StationReferencePlatformNumberFromDurationMinutes  

 TrainType format:  
 COMMUTER | FREIGHT | IC | URBAN  

 TrainComposition format:  
 vvv...  
 Each v represents one vehicle. L = locomotive (or control post), C = cargo car, P = passenger car  
 
 Flags format:  
 ff  
 Each f is one flag. 0 = flag not set, 1 = flag set, X = position not used  
 Flag positions:  
 1 unused (X)  
 2 NoBrakingPenalization - if set (1), train does NOT receive penalization when braking at signals  


**例子**  
|列车编号|类型|最高时速|动拖布置|flag|车站1|车站2|车站3|   
|--|--|--|--|--|--|--|--|  
|C5627| COMMUTER| 200| LPPL| X1 :| b#1#07:03:00#0| a#3#07:06:00#20 |c#1#07:29:00#0 |  



**车次相关信息**  
    车次信息不变 默认commuter  
    依照类型设置最高速度 D->200 G->300  
    车辆细节 先默认 MTTM 后续修改长编或者重连?  
  

**停站相关信息**  
    仅三点式 进场车站 停站 离场车站  
    车站 筛选所有车次始发终到，设置字典更改进场离场  
    股道指定 随机指定？  
  

**表格信息**  
从路路通截图导出为Excel进行处理  
![格式](表格格式.png)  
始发站和终到站更换为进场车站和离场车站  
  

**车站信息**  
![车站](鄂州站.png)  
单个车站及进场离场布置  
  

**库函数部分及常规列车信息部分**  
导入库函数，处理表格以及替换字符并导出  
使用字典建立常规信息的映射关系  

In [22]:
import numpy
import pandas
import datetime
import random

Excelpath = "鄂州站时刻表.xlsx"  # Excel时刻表文件路径
TextPath="trains鄂州.txt" #游戏时刻表文件路径
species = {'K': ['120','LPPP','普速'], 'T': ['140','LPPP','普速'], 'Z': ['160','LPPP','普速'],
         'D': ['200','LPPL','动车'], 'C': ['200','LPPL','动车'], 'G': ['300','LPLLPL','高速']}  # 速度和编组映射关系
# 车站-编号,股道,股道到达及用时映射关系
# 股道到达映射关系.可到达股道,图片左右侧线路key值相同则掉向,
station = {'武九线黄石方向': ['b', '123', 0, 3], '武九线何刘方向': ['e', '34', 1, 10], '鄂州站': ['a', '123456', -1, 0], 
           '武九客专黄石北方向': ['c', '5', 0, 3], '武九客专葛店南方向': ['d', '6', 1, 8], }
ThisStation = '鄂州站'




# track = {}
# 车型关系--待筛选，默认短编
marshalling = {}

# 始发终到映射关系--待筛选
dst = {}

# 股道到达,图片左右侧线路,,key值不同则掉向
# turnst = {'武九客专武汉方向': 0, '武九客专鄂州方向': 1, '武冈城际黄冈方向': 1}

trainDF = pandas.DataFrame(
    columns=['列车编号', '类型', '最高时速', '动拖布置', 'flag'])  # 列车整体信息
arriveStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 进场信息
stopStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 停站信息
leaveStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 离场信息

# 存储字符串形式的最终结果
trainList = []
arriveStList = []
stopStList = []
leaveStList = []


**读取表格及数据处理部分**  
>1,读取表格并去除空行  
>2,从车次提取速度等级  
>3,筛选始发站终到站建立替换字典  
  
  
**车次字符串生成部分**
>1,车次信息部分
>>车次号,类型,动拖布置及flag保持默认  
>>速度等级依照KTZDG等区分映射  

>2,停站部分  
>>停站2(主要车站) 到时及停站时间来自表格部分  股道随机  
>>进场及离场部分 依照游戏先期测试进行平移推算  股道依照上下行安排  

In [23]:
# 读取文件
sheet = pandas.read_excel(io=Excelpath)

sheet = sheet.dropna()
sheet=sheet.reset_index(drop=True)

print(sheet.head())

trainInfo = sheet["车次名称"].to_frame()  # 车辆信息
#统计始发终到车站信息
ts=pandas.concat([sheet["始发站"].value_counts(),sheet["终到站"].value_counts()])
totalStation=ts.index

print(set(list(totalStation))) #所有始发站和终到站统计

    车次名称        到时        开时  始发站  终到站
0  C5051  09:59:00  10:01:00  咸宁南   阳新
1  C5522  20:52:00  21:00:00  大冶北   武汉
2  C5604  07:22:00  07:24:00  黄石北   武汉
3  D2175  10:09:00  10:16:00  宜昌东  杭州西
4  D2225  16:17:00  16:24:00  重庆北   福州
{'兰州西', '赣州西', '汉口', '福州', '咸宁南', '郑州东', '武昌', '西安北', '南昌', '杭州西', '宁波', '成都', '厦门北', '武汉东', '宜昌东', '云梦东', '深圳东', '阳新', '景德镇北', '重庆北', '黄石北', '十堰', '梅州西', '上海虹桥', '大冶北', '仙桃', '杭州', '北京西', '温州', '深圳北', '南昌西', '上海南', '温州南', '武汉', '襄阳'}


In [24]:
# 手动建立映射关系
ArrLeaveSt = {'上海南':'武九线黄石方向',
 '上海虹桥':'武九客专黄石北方向',
 '云梦东':'武九客专葛店南方向',
 '仙桃':'武九客专葛店南方向',
 '兰州西':'武九客专葛店南方向',
 '北京西':'武九客专葛店南方向',
 '十堰':'武九线何刘方向',
 '南昌':'武九线黄石方向',
 '南昌西':'武九客专黄石北方向',
 '厦门北':'武九客专黄石北方向',
 '咸宁南':'武九客专葛店南方向',
 '大冶北':'武九客专黄石北方向',
 '宁波':'武九线黄石方向',
 '宜昌东':'武九客专葛店南方向',
 '成都':'武九线何刘方向',
 '景德镇北':'武九客专黄石北方向',
 '杭州':'武九线黄石方向',
 '杭州西':'武九客专黄石北方向',
 '梅州西':'武九客专黄石北方向',
 '武昌':'武九线何刘方向',
 '武汉':'武九客专葛店南方向',
 '武汉东':'武九客专葛店南方向',
 '汉口':'武九客专葛店南方向',
 '深圳东':'武九线黄石方向',
 '深圳北':'武九客专黄石北方向',
 '温州':'武九线黄石方向',
 '温州南':'武九客专黄石北方向',
 '福州':'武九客专黄石北方向',
 '襄阳':'武九客专葛店南方向',
 '西安北':'武九客专葛店南方向',
 '赣州西':'武九客专葛店南方向',
 '郑州东':'武九客专葛店南方向',
 '重庆北':'武九客专葛店南方向',
 '阳新':'武九客专黄石北方向',
 '黄石北':'武九客专黄石北方向'}


ArrLeaveSt 


{'上海南': '武九线黄石方向',
 '上海虹桥': '武九客专黄石北方向',
 '云梦东': '武九客专葛店南方向',
 '仙桃': '武九客专葛店南方向',
 '兰州西': '武九客专葛店南方向',
 '北京西': '武九客专葛店南方向',
 '十堰': '武九线何刘方向',
 '南昌': '武九线黄石方向',
 '南昌西': '武九客专黄石北方向',
 '厦门北': '武九客专黄石北方向',
 '咸宁南': '武九客专葛店南方向',
 '大冶北': '武九客专黄石北方向',
 '宁波': '武九线黄石方向',
 '宜昌东': '武九客专葛店南方向',
 '成都': '武九线何刘方向',
 '景德镇北': '武九客专黄石北方向',
 '杭州': '武九线黄石方向',
 '杭州西': '武九客专黄石北方向',
 '梅州西': '武九客专黄石北方向',
 '武昌': '武九线何刘方向',
 '武汉': '武九客专葛店南方向',
 '武汉东': '武九客专葛店南方向',
 '汉口': '武九客专葛店南方向',
 '深圳东': '武九线黄石方向',
 '深圳北': '武九客专黄石北方向',
 '温州': '武九线黄石方向',
 '温州南': '武九客专黄石北方向',
 '福州': '武九客专黄石北方向',
 '襄阳': '武九客专葛店南方向',
 '西安北': '武九客专葛店南方向',
 '赣州西': '武九客专葛店南方向',
 '郑州东': '武九客专葛店南方向',
 '重庆北': '武九客专葛店南方向',
 '阳新': '武九客专黄石北方向',
 '黄石北': '武九客专黄石北方向'}

In [25]:
#'列车编号','类型','最高时速','动拖布置','flag' 列车整体信息

for index,row in trainInfo.iterrows():
    trainNum=row["车次名称"]
    maxspeed=(species.get(trainNum[0]))[0]#有字头的列车
    if maxspeed==None: #最高速度未找到
        maxspeed='120' #普客
    
    marshalling=(species.get(trainNum[0]))[1]
    #trainTC=marshalling.get() #动拖布置
    trainDF.loc[index]=[trainNum,"COMMUTER",maxspeed,marshalling,"X1"]
    trainList.append("{0} {1} {2} {3} {4} : ".format(trainNum,"COMMUTER",maxspeed,marshalling,"X1"))
trainDF.head()

,列车编号,类型,最高时速,动拖布置,flag
0,C5051,COMMUTER,200,LPPL,X1
1,C5522,COMMUTER,200,LPPL,X1
2,C5604,COMMUTER,200,LPPL,X1
3,D2175,COMMUTER,200,LPPL,X1
4,D2225,COMMUTER,200,LPPL,X1


In [26]:
# arriveStList=pandas.DataFrame(columns=['车站名称','股道','到达时间','停站时间'])#进场信息
# stopiveStList=pandas.DataFrame(columns=['车站名称','股道','到达时间','停站时间'])#停站信息
# leaveStList=pandas.DataFrame(columns=['车站名称','股道','到达时间','停站时间'])#离场信息
# stationInfo = sheet[["到时", "开时","始发站", "终到站"]]  # 时间及停站信息

for index, row in sheet.iterrows():
    arriveSt = ArrLeaveSt.get(row["始发站"])
    leaveSt = ArrLeaveSt.get(row["终到站"])

    # 上下行编号区分车辆进场股道为1,2道

    if int(row["车次名称"].strip()[-1]) % 2 == 0:
        arriveTrack = 2  # 偶数位上行车
    else:
        arriveTrack = 1  # 奇数下行
    # 依照掉向区分离场股道 数值相同即为同一侧需掉向，不同即为不掉向顺向行驶
    if (station.get(arriveSt))[2] == (station.get(leaveSt))[2]:
        leaveTrack = 3-arriveTrack  # 2->1,1->2
    else:
        leaveTrack = arriveTrack

    # 停站股道,依照映射随机选择
    stopTrack = random.choice((station.get(arriveSt)[1]))

    at=row["到时"]
    lt=row["开时"]
    # 统一时间格式
    strTime1 = datetime.datetime.strptime(str(at), "%H:%M:%S")
    strTime2 = datetime.datetime.strptime(str(lt), "%H:%M:%S")

    stopTime = (strTime2-strTime1).seconds/60  # 分钟为单位的停站时间

    arrTime1=strTime1-datetime.timedelta(minutes=(station.get(arriveSt))[3])
    #print((str(arrTime1))[10,-1])
    arrTime2=str(arrTime1.strftime('%Y-%m-%d %H:%M:%S'))[-8:]
    arrTime=datetime.datetime.strptime(arrTime2, "%H:%M:%S")

    leaveTime1=strTime2+datetime.timedelta(minutes=(station.get(leaveSt))[3])
    leaveTime2=str(leaveTime1.strftime('%Y-%m-%d %H:%M:%S'))[-8:]
    leaveTime=datetime.datetime.strptime(leaveTime2, "%H:%M:%S")

    # 进场
    arriveStDF.loc[index] = [arriveSt, arriveTrack, arrTime2, 0]
    arriveStList.append("{0}#{1}#{2}#{3}".format(
        (station.get(arriveSt))[0], arriveTrack, arrTime2, 0))
    # 停站
    stopStDF.loc[index] = [ThisStation, stopTrack, row["到时"], stopTime]
    stopStList.append("{0}#{1}#{2}#{3}".format(
        (station.get(ThisStation))[0], stopTrack, row["到时"], int(stopTime)))

    # 离场
    leaveStDF.loc[index] = [leaveSt, leaveTrack, leaveTime2, 0]
    leaveStList.append("{0}#{1}#{2}#{3}".format(
        (station.get(leaveSt))[0], leaveTrack, leaveTime2, 0))



In [27]:
print(arriveStDF.head(),'\n',stopStDF.head(),'\n',leaveStDF.head())

        车站名称  股道      到达时间  停站时间
0  武九客专葛店南方向   1  09:51:00     0
1  武九客专黄石北方向   2  20:49:00     0
2  武九客专黄石北方向   2  07:19:00     0
3  武九客专葛店南方向   1  10:01:00     0
4  武九客专葛店南方向   1  16:09:00     0 
   车站名称 股道      到达时间  停站时间
0  鄂州站  6  09:59:00   2.0
1  鄂州站  5  20:52:00   8.0
2  鄂州站  5  07:22:00   2.0
3  鄂州站  6  10:09:00   7.0
4  鄂州站  6  16:17:00   7.0 
         车站名称  股道      到达时间  停站时间
0  武九客专黄石北方向   1  10:04:00     0
1  武九客专葛店南方向   2  21:08:00     0
2  武九客专葛店南方向   2  07:32:00     0
3  武九客专黄石北方向   1  10:19:00     0
4  武九客专黄石北方向   1  16:27:00     0


**最终车次结果**  
合并车辆信息和停站信息  
  
**导出部分**  
导出为train.txt手动附加原文件头部之后替换原时刻表文件  

In [28]:
timeTable =open(file=TextPath,mode="a")
finalRes=[]
for i in range(0,len(arriveStList)):
    tstr="{0}{1} {2} {3} ".format(trainList[i],arriveStList[i],stopStList[i],leaveStList[i])
    finalRes.append(tstr)
    tstr1=tstr+'\n'
    timeTable.writelines(tstr1)

timeTable.close()
finalRes


['C5051 COMMUTER 200 LPPL X1 : d#1#09:51:00#0 a#6#09:59:00#2 c#1#10:04:00#0 ',
 'C5522 COMMUTER 200 LPPL X1 : c#2#20:49:00#0 a#5#20:52:00#8 d#2#21:08:00#0 ',
 'C5604 COMMUTER 200 LPPL X1 : c#2#07:19:00#0 a#5#07:22:00#2 d#2#07:32:00#0 ',
 'D2175 COMMUTER 200 LPPL X1 : d#1#10:01:00#0 a#6#10:09:00#7 c#1#10:19:00#0 ',
 'D2225 COMMUTER 200 LPPL X1 : d#1#16:09:00#0 a#6#16:17:00#7 c#1#16:27:00#0 ',
 'D2226 COMMUTER 200 LPPL X1 : c#2#14:43:00#0 a#5#14:46:00#2 d#2#14:56:00#0 ',
 'D2231 COMMUTER 200 LPPL X1 : d#1#15:39:00#0 a#6#15:47:00#2 c#1#15:52:00#0 ',
 'D2232 COMMUTER 200 LPPL X1 : c#2#14:20:00#0 a#5#14:23:00#3 d#2#14:34:00#0 ',
 'D2376 COMMUTER 200 LPPL X1 : c#2#15:03:00#0 a#5#15:06:00#2 d#2#15:16:00#0 ',
 'D3223 COMMUTER 200 LPPL X1 : d#1#17:48:00#0 a#6#17:56:00#2 c#1#18:01:00#0 ',
 'D3226 COMMUTER 200 LPPL X1 : c#2#16:01:00#0 a#5#16:04:00#2 d#2#16:14:00#0 ',
 'D3251 COMMUTER 200 LPPL X1 : d#1#17:26:00#0 a#6#17:34:00#2 c#1#17:39:00#0 ',
 'D3256 COMMUTER 200 LPPL X1 : c#2#17:19:00#0 a#5#17